# Entrega del modelo (local)

In [ ]:
import mlflow
import pandas as pd

Para servir el modelo en local con `MLflow` se puede seguir [esta guía](https://mlflow.org/docs/latest/deployment/deploy-model-locally.html). Es importante que se inicie el `tracking` de los recursos de `MLflow` en el directorio adecuado (la carpeta `mlflow`, donde están las carpetas de `mlrun` y `mlartifacts`) y que se inicie la interfaz gráfica (`mlflow ui`) para que se puedan cargar los modelos.

In [ ]:
%%bash
if [ pwd != 'C:\Users\valde\Desktop\Airbnb_Madrid\mlflow' ]
then
    cd mlflow/
fi

Primero se comprueba que se puede cargar el modelo de `MLflow` especificando el nombre y su versión y realizando predicciones `offline` por lotes:

In [ ]:
# URL donde se trackea el registro de recursos
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Para cargar el modelo hace falta especificar una de estas dos opciones:
# - 'runs:/<run_id>/<artifact_path>'
# - 'models:/<model_name>/<model_version>'
logged_model = 'models:/decision_tree/1'

# Este comando es por si acaso para instalar las dependencias del modelo especificado con 'pip'
mlflow.pyfunc.get_model_dependencies(logged_model, 'pip')

# Carga el modelo de 'Decision Tree' como objeto 'PyFuncModel'
loaded_model = mlflow.sklearn.load_model(logged_model)

# Predicción con un ejemplo 'raw' (el formato de entrada es un 'dataframe', ya sea de una fila o N filas)
loaded_model.predict(pd.DataFrame(columns = ["square_feet", "accommodates", "cleaning_fee", "beds", "availability_30", "bedrooms", "availability_60", "availability_90", "availability_365", "security_deposit", "bathrooms", "guests_included", "neighbourhood", "neighbourhood_group_cleansed", "property_type", "room_type", "amenities", "X", "Y", "Z", "date"],
                                  data= [[5.46, 6, 60.0, 4.0, 30, 2.0, 60, 88, 359, 600.0, 2.0, 4, "Retiro", "Retiro", "Apartment", "Entire home/apt", 18, 4846242.12107298, -312605.06587381434, 4134833.3663519137, 2]]))

Ahora se prueba a servir el modelo con `Flask` en un `endpoint` de un `servicio web` local en el `puerto 1234`:

In [ ]:
# Se establece la variable de entorno para trackear la URL donde el Registro de modelos reside
%env MLFLOW_TRACKING_URI=http://localhost:5000

# Se sirve el modelo de producción desde el Registro de modelos, con un puerto distinto al que usa MLflow sin entorno Conda
!mlflow models serve -m "models:/decision_tree/1" -p 1234 #--enable-mlserver

El servidor de inferencia tiene 4 `endpoints`:

- `/invocations`: Un `endpoint` que acepta peticiones con datos de entrada y que devuelve las predicciones (`POST`).
- `/ping`: Para realizar comprobaciones de funcionamiento (`GET`).
- `/health`: Lo mismo que en el caso anterior (`GET`).
- `/version`: Devuelve la versión de `MLflow` (`GET`).

La celda e código anterior está permanentemente en ejecución, ya que el modelo está siendo `servido` en el `puerto 1234` del `localhost`, es decir, en la URL `http://127.0.0.1:1234/`. Por tanto, ahora se puede hacer un `curl` a esta dirección para hacer predicciones o ejecutar un `script` de Python para realizar la inferencia con una solicitud `POST` a la `URL` especificada anteriormente. El `script` sería algo parecido a esto:

In [ ]:
# Librerías
import requests
import json
import numpy as np

# URL local donde se sirven las predicciones del modelo
url = 'http://127.0.0.1:1234/invocations'

# Header para contenido de tipo JSON
headers = {'Content-type': 'application/json'}

# Datos a los que realizar la inferencia. MUY IMPORTANTE: Seguir el esquema de 'dataframe_split' con 'columns' y 'data'
X_test = json.dumps({
    "dataframe_split": {
        "columns": ["square_feet", "accommodates", "cleaning_fee", "beds", "availability_30", "bedrooms", "availability_60", "availability_90", "availability_365", "security_deposit", "bathrooms", "guests_included", "neighbourhood", "neighbourhood_group_cleansed", "property_type", "room_type", "amenities", "X", "Y", "Z", "date"], 
        "data": [
            [np.nan, 6, 60.0, 4.0, 30, 2.0, 60, 88, 359, 600.0, 2.0, 4, "Retiro", "Retiro", "Apartment", "Entire home/apt", 18, 4846242.12107298, -312605.06587381434, 4134833.3663519137, 2], 
            [np.nan, 6, 60.0, 4.0, 30, 2.0, 60, 88, 359, 600.0, 2.0, 4, "Retiro", "Retiro", "Apartment", "Entire home/apt", 18, 4846242.12107298, -312605.06587381434, 4134833.3663519137, 2]
            ]
    }
})                          

# Solicitud 'POST' al puerto donde el modelo está siendo expuesto con el 'header' de contenido JSON
resp = requests.post(url, headers=headers, data = X_test)

# Imprime el contenido de la respuesta
print(resp.content)

Una alternativa a ejecutar el código de `Python` anterior es realizar la inferencia con un comando `curl` al `localhost`:

Como se puede apreciar, no se trata de un método amigable para realizar las predicciones, y por tanto, crear una imagen de `Docker` y exportar a producción. Por lo tanto, se va a usar otra alternativa.

# Despliegue (producción)

## Docker

De esta forma, dentro de la carpeta `app` se ha creado una `API` con `FastAPI` que va a servir el modelo y una aplicación web con `Streamlit` para tener una interfaz gráfica que haga peticiones `POST` a la API, por lo que así se despliega el modelo de forma más amigable mediante dicha API y la interfaz gráfica. Una vez se ha creado el código de la API y la aplicación y han sido testeadas en local, se crean las imágenes de `Docker` mediante archivo `Dockerfile`:
- Se copia el código de la API o de la aplicación web de `Streamlit` (dependiendo del `Dockerfile` que se esté creando), el archivo `requirements.txt` con las librerías requeridas y el modelo guardado (solo para el `Dockerfile` de la API, que será quien sirva el modelo) al directorio de trabajo de la imagen de `Docker`.
- Se actualizan los paquetes e instalan las librerías requeridas del `requirements.txt`.
- Se expone el puerto `8000` (para el `Dockerfile` de la `API`) y el `8501` (para el `Dockerfile` de la aplicación web), ya que posteriormente se van a usar para exponer la API y la aplicación web, respectivamente.
- Se ejecuta la aplicación web de `FastAPI` (`server.py`) y de `Streamlit` (`app.py`) especificando la `IP` y el `puerto` donde se servirá.

Tras crear estos dos archivos y configurarlo correctamente, se crean las imágenes con el siguiente comando (se usa de ejemplo la imagen `backend` de la `API`):

<p style="text-align: center;">  docker build -f .\app\fullstack\backend\Dockerfile -t app-backend-airbnb:v1.1 .  </p>

Mediante el argumento `-t` se especifica el nombre y el `tag` de la imagen en formato `nombre:tag`; y mediante el argumento `-f` el archivo `Dockerfile` a usar. Por lo que la imagen creada se llama `app-backend-airbnb` y el `tag` es `v1.1` a partir del `Dockerfile` de la carpeta `fullstack\backend`. El `.` del final indica el contexto del `build`, es decir, el directorio raíz a partir de donde `Docker` buscará los archivos (en este caso, la carpeta raíz del proyecto `Airbnb_Madrid`).

Una vez se crea la imagen de `Docker`, ya se puede consultar la API o la aplicación web dentro de un contenedor. Para ello se inicia uno de estos contenedores, como se ha comentado, a partir de la imagen creada, con un comando como el siguiente (se pone como ejemplo de nuevo el caso de la imagen `backend` de `FastAPI`):

<p style="text-align: center;"> docker container run -itd -p 8000:8000 --name backend-fastapi 3ed </p>

Mediante este comando se inicia un contenedor a partir de la imagen creada, enlazando el puerto `8000` de nuestro `localhost` con el puerto `8000` del contenedor (el puerto donde se encuentra la `API` dentro del contenedor). De esta forma, si se abre en un navegador la dirección `127.0.0.1:8000` (el `localhost`) se accederá a la `API` de `FastAPI`, ya que el puerto `8000` de nuestro `localhost` está conectado con el puerto `8000` del contenedor, que es donde se aloja la `API`.

Después de construir la imagen de Docker, ésta se sube a `Dockerhub` para tenerlo almacenado en un repositorio y poder descargar la imagen posteriormente en cualquier entorno. Para ello, primero se crea un `tag` a la imagen de `Docker` creada para especificar cual es nuestro usuario `Dockerhub` y cual sería el nombre de la imagen y el `tag` en el repositorio. Por tanto, se ejecuta el siguiente comando (se pone como ejemplo de nuevo el caso de la imagen `backend` de `FastAPI`):

<p style="text-align: center;"> docker image tag app-backend-airbnb:v1.1 xxxx/app-backend-airbnb:v1.1 </p>

Donde `xxxx` es el nombre de la cuenta de `Dockerhub`.

Una vez la imagen ya ha sido `taggeada`, se sube al repositorio de `Dockerhub`:

<p style="text-align: center;"> docker image push xxxx/app-backend-airbnb:v1.1 </p>

## Kubernetes

`Kubernetes` es un sistema de orquestación de contenedores usado para el despliegue de contenedores de `Docker`. Está diseñado para administrar eficientemente y coordinar `clusters` y cargas de trabajo a gran escala en un entorno de producción. Además, ayuda a gestionar servicios contenerizados mediante la automatización en el despliegue.

De esta forma, se crea un despliegue con, por ejemplo, tres `pods`, cada uno de los cuales tendrá dos contenedores (uno proveniente de la imagen de la aplicación web; y otro proveniente de la imagen de la API). Aunque si es necesario, se puede escalar dicho despliegue a más o menos `pods` dependiendo del tráfico que requiera la API y la aplicación web. Además, estos `pods` se van a exponer a través de un servicio de tipo `LoadBalancer`, de forma que la API y la aplicación web se hacen accesibles a clientes externos que están fuera del `cluster` de `Kubernetes` y además añade funcionalidad de balanceo de carga para distribuir el tráfico entre los distintos `pods`, reduciendo los efectos negativos de sobrecarga.

Para ello se crea un archivo `YAML` que define tanto el servicio como el despliegue:

Como se puede apreciar, los puertos de los contenedores deben ser el `8000` para la API y el `8501` para la aplicación web, que es donde están alojadas la API y la aplicación web en cada una de las dos imágenes, y por tanto, los `targetPort` del servicio también debe ser los mismos, ya que estos serán los puertos a los que va a apuntar el servicio `LoadBalancer`.

En el cluster local de `Minikube`, una vez creado el despliegue y el servicio, para acceder a la API y a la aplicación web hay que ejecutar el siguiente comando que devuelve dos `URLs` (una para la API y otra para la aplicación web) para conectarse al servicio que expone dichas aplicaciones:

<p style="text-align: center;"> minikube -n airbnb service app-fullstack-airbnb-service </p>

Esto lo que hace es crear un túnel para el servicio `app-fullstack-airbnb-service`, creando un movimiento de datos de una red a otra. Es decir, se mueve el flujo de datos de la `IP` y de cada uno de los puertos del `nodo` de `Minikube` (`nodePort`) que usa el servicio, a la `IP` local y a dos puertos aleatorios del `localhost`, de forma que se puede consultar tanto la `API` como la aplicación web en el navegador local.